## Анализ базы данных книжного магазина

**Цель**: изучение базы данных книжного магазина

**Задачи**:
- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
- Определить автора с самой высокой средней оценкой книг — с учетом книг с 50 и более оценками;
- Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [73]:
# pip install psycopg2-binary

In [74]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'user', # имя пользователя
'pwd': 'pass', # пароль
'host': 'host',
'port': port, # порт подключения
'db': 'db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 


## Предобработка данных

In [75]:
def check(data):
    data.info()
    data.head(5)
    data.isnull().sum()
    data.duplicated().sum()
    return display(data.head(5)),\
print('Количество пропусков:', data.isnull().sum()),\
print('Количество дубликатов:', data.duplicated().sum(),\
             )

In [76]:
query = '''
    SELECT * 
    FROM books
'''

check(pd.io.sql.read_sql(query, con = engine))



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Количество пропусков: book_id             0
author_id           0
title               0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64
Количество дубликатов: 0


(None, None, None)

In [77]:
query = '''
    SELECT * 
    FROM authors
'''

check(pd.io.sql.read_sql(query, con = engine))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Количество пропусков: author_id    0
author       0
dtype: int64
Количество дубликатов: 0


(None, None, None)

In [78]:
query = '''
    SELECT * 
    FROM publishers
'''

check(pd.io.sql.read_sql(query, con = engine))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Количество пропусков: publisher_id    0
publisher       0
dtype: int64
Количество дубликатов: 0


(None, None, None)

In [79]:
query = '''
    SELECT * 
    FROM ratings
'''

check(pd.io.sql.read_sql(query, con = engine))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Количество пропусков: rating_id    0
book_id      0
username     0
rating       0
dtype: int64
Количество дубликатов: 0


(None, None, None)

In [86]:
query = '''
    SELECT * 
    FROM reviews
'''

check(pd.io.sql.read_sql(query, con = engine))
print('Количество ID книг в базе с отзывами:', len(pd.io.sql.read_sql(query, con = engine)['book_id'].unique()))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Количество пропусков: review_id    0
book_id      0
username     0
text         0
dtype: int64
Количество дубликатов: 0
Количество ID книг в базе с отзывами: 994


Пропусков и дубликатов во всех датасетах нет.

**Задача 1**: Посчитать сколько книг вышло после 1 января 2000 года

In [81]:
query = '''
    SELECT
        COUNT(title) AS title_count
    FROM books
    WHERE CAST(publication_date AS timestamp) > '2000-01-01' 
'''

pd.io.sql.read_sql(query, con = engine)

,title_count
0,819


**Задача 2**: Для каждой книги посчитать количество обзоров и среднюю оценку.

In [87]:
query = '''
    SELECT 
        b.book_id,
        b.title,
        COUNT(DISTINCT rev.review_id) AS review_count,
        AVG(rate.rating) AS rating_mean
    FROM books AS b
    LEFT JOIN ratings AS rate ON rate.book_id = b.book_id
    LEFT JOIN reviews AS rev ON rev.book_id = b.book_id
    GROUP BY b.book_id
    ORDER BY b.book_id
'''

display(pd.io.sql.read_sql(query, con = engine))

print('Сумма ревью на книги издательства', pd.io.sql.read_sql(query, con = engine)['review_count'].sum())


,book_id,title,review_count,rating_mean
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,997,Xenocide (Ender's Saga #3),3,3.400000
997,998,Year of Wonders,4,3.200000
998,999,You Suck (A Love Story #2),2,4.500000


Сумма ревью на книги издательства 2793


В результате у нас количество тайтлов снизилось до 994 (с 1000 изначального датасета.) Связано это с тем, что в датасете с ревью отзывы оставлены на 994 тайтла.

**Задача 3**: Определить издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [83]:
query = '''
    SELECT 
        p.publisher,
        COUNT(DISTINCT b.title) AS title_count
    FROM publishers AS p
    LEFT JOIN books AS b ON b.publisher_id = p.publisher_id
    WHERE b.num_pages > 50
    GROUP BY p.publisher
    ORDER BY title_count DESC
    LIMIT 1;

'''

pd.io.sql.read_sql(query, con = engine)

,publisher,title_count
0,Penguin Books,42


**Задача 4**: Определить автора с самой высокой средней оценкой книг — с учетом книг с 50 и более оценками.

In [84]:
query = '''
    SELECT
        a.author,
        AVG(rate.rating) AS rating_books_mean
    FROM authors AS a
    LEFT JOIN books AS b ON b.author_id = a.author_id
    LEFT JOIN ratings AS rate ON rate.book_id = b.book_id
    WHERE b.title IN (SELECT 
                        b.title
                    FROM books AS b
                    LEFT JOIN ratings AS rate ON rate.book_id = b.book_id
                    GROUP BY b.title
                    HAVING COUNT(rate.rating_id) > 50)
    GROUP BY a.author
    ORDER BY rating_books_mean DESC
    LIMIT 1

'''

pd.io.sql.read_sql(query, con = engine)

,author,rating_books_mean
0,J.K. Rowling/Mary GrandPré,4.287097


**Задача 5**: Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [85]:
query = '''
SELECT AVG(review_sum) AS review_over_50_mean
    FROM (SELECT
                        COUNT(rev.review_id) AS review_sum
                        FROM reviews AS rev
                        WHERE rev.username IN (SELECT 
                                                rate.username
                                                FROM ratings AS rate
                                                GROUP BY rate.username
                                                HAVING COUNT(rate.rating_id) > 50)
                        GROUP BY rev.username) AS review_sum
        

'''

pd.io.sql.read_sql(query, con = engine)

,review_over_50_mean
0,24.333333


## Общие выводы

С начала 2020-го года издательство выпустило 820 книг. Всего в каталоге 999 оригинальных наименований, к каждому из которых мы посчитали средние оценки читателей и количество рецензий. Из всех издательств больше всего книг выпустило Penguin Books (42 наименования.) Самый популярный массовый автор - J.K. Rowling, у ее произведений самая высокая средняя оценка - 4,3. Самые активные читатели в среднем проставляют 24,3 оценки. 